<a href="https://colab.research.google.com/github/a42437-ml/master_thesis_files/blob/main/qos_files/sip_rtp_setup_rtt_jitter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install Wireshark's tshark CLI (needed by pyshark)
!apt-get update
!apt-get install -y tshark

# Install pyshark (Python wrapper for tshark)
!pip install pyshark

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,840 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [4,932 kB]
Get:13 http://security.ubuntu.com/ub

In [ ]:
!pip install pyshark

import pyshark
import subprocess
import json

pcap_file = 'sip_rtp_capture_tc-07 (1).pcap'

try:
    # Use tshark directly to avoid async issues
    cmd = [
        'tshark',
        '-r', pcap_file,
        '-Y', 'sip',
        '-T', 'json',
        '-e', 'sip.Call-ID',
        '-e', 'sip.Method',
        '-e', 'sip.Status-Code',
        '-e', 'frame.time_epoch'
    ]

    result = subprocess.run(cmd, capture_output=True, text=True)

    if result.returncode != 0:
        print(f"tshark error: {result.stderr}")
    else:
        packets = json.loads(result.stdout)

        call_times = {}
        setup_durations = []

        for pkt in packets:
            layers = pkt.get('_source', {}).get('layers', {})

            if 'sip.Call-ID' in layers:
                call_id = layers['sip.Call-ID'][0] if isinstance(layers['sip.Call-ID'], list) else layers['sip.Call-ID']
                timestamp = float(layers['frame.time_epoch'][0]) if isinstance(layers['frame.time_epoch'], list) else float(layers['frame.time_epoch'])

                # Check for INVITE
                if 'sip.Method' in layers and layers['sip.Method'][0] == 'INVITE':
                    if call_id not in call_times:
                        call_times[call_id] = {'INVITE': timestamp}

                # Check for 200 OK
                elif 'sip.Status-Code' in layers and layers['sip.Status-Code'][0] == '200':
                    if call_id in call_times:
                        call_times[call_id]['200OK'] = timestamp

        # Process results
        for call_id, times in call_times.items():
            if 'INVITE' in times and '200OK' in times:
                duration = times['200OK'] - times['INVITE']
                setup_durations.append(duration)
                print(f"Call-ID: {call_id} | Setup Time: {duration:.3f} seconds")

        if setup_durations:
            avg_time = sum(setup_durations) / len(setup_durations)
            print(f"\n✅ Analyzed {len(setup_durations)} calls")
            print(f"📊 Average Setup Time: {avg_time:.3f} seconds")
        else:
            print("⚠️ No complete call setups found.")

except Exception as e:
    print(f"Error: {e}")

Call-ID: 1-107395@38.242.140.216 | Setup Time: 10.056 seconds
Call-ID: 2-107395@38.242.140.216 | Setup Time: 10.055 seconds
Call-ID: 3-107395@38.242.140.216 | Setup Time: 10.159 seconds
Call-ID: 4-107395@38.242.140.216 | Setup Time: 10.060 seconds
Call-ID: 5-107395@38.242.140.216 | Setup Time: 10.078 seconds
Call-ID: 91bbd581-dab0-123e-968f-00505658449a | Setup Time: 10.097 seconds
Call-ID: 91bc3db0-dab0-123e-968f-00505658449a | Setup Time: 10.052 seconds
Call-ID: 91bc73dd-dab0-123e-968f-00505658449a | Setup Time: 10.164 seconds
Call-ID: 91bcd6b1-dab0-123e-968f-00505658449a | Setup Time: 10.091 seconds
Call-ID: 91bd8f3b-dab0-123e-968f-00505658449a | Setup Time: 10.157 seconds
Call-ID: 6-107395@38.242.140.216 | Setup Time: 10.158 seconds
Call-ID: 7-107395@38.242.140.216 | Setup Time: 10.197 seconds
Call-ID: 8-107395@38.242.140.216 | Setup Time: 10.153 seconds
Call-ID: 9-107395@38.242.140.216 | Setup Time: 10.119 seconds
Call-ID: 97c8a5e8-dab0-123e-968f-00505658449a | Setup Time: 10.082 

In [ ]:
import pyshark
import nest_asyncio
nest_asyncio.apply()

pcap_file = 'sip_rtp_capture_tc-07.01.pcap'

try:
    cap = pyshark.FileCapture(pcap_file, display_filter='sip')

    call_times = {}
    setup_durations = []

    for pkt in cap:
        try:
            if hasattr(pkt, 'sip'):
                call_id = pkt.sip.call_id
                timestamp = float(pkt.sniff_timestamp)

                # Check for INVITE method
                if hasattr(pkt.sip, 'method') and pkt.sip.method == 'INVITE':
                    if call_id not in call_times:
                        call_times[call_id] = {'INVITE': timestamp}

                # Check for 200 OK response - but only if we haven't already recorded it
                elif (hasattr(pkt.sip, 'status_code') and pkt.sip.status_code == '200' and
                      call_id in call_times and '200OK' not in call_times[call_id]):
                    # Additional check: make sure this 200 OK comes reasonably soon after INVITE
                    # (not after a BYE which would be much later)
                    if timestamp - call_times[call_id]['INVITE'] < 30:  # Within 30 seconds
                        call_times[call_id]['200OK'] = timestamp

        except AttributeError:
            continue

    # Process results
    for call_id, times in call_times.items():
        if 'INVITE' in times and '200OK' in times:
            duration = times['200OK'] - times['INVITE']
            setup_durations.append(duration)
            print(f"Call-ID: {call_id} | Setup Time: {duration:.3f} seconds")

    if setup_durations:
        avg_time = sum(setup_durations) / len(setup_durations)
        print(f"\nAnalyzed {len(setup_durations)} calls")
        print(f"Average Setup Time: {avg_time:.3f} seconds")
        print(f"Min Setup Time: {min(setup_durations):.3f} seconds")
        print(f"Max Setup Time: {max(setup_durations):.3f} seconds")
    else:
        print("⚠️ No complete call setups found.")

    cap.close()

except Exception as e:
    print(f"Error: {e}")

Call-ID: 1-107395@38.242.140.216 | Setup Time: 0.037 seconds
Call-ID: 2-107395@38.242.140.216 | Setup Time: 0.038 seconds
Call-ID: 3-107395@38.242.140.216 | Setup Time: 0.048 seconds
Call-ID: 4-107395@38.242.140.216 | Setup Time: 0.041 seconds
Call-ID: 5-107395@38.242.140.216 | Setup Time: 0.042 seconds
Call-ID: 91bbd581-dab0-123e-968f-00505658449a | Setup Time: 0.004 seconds
Call-ID: 91bc3db0-dab0-123e-968f-00505658449a | Setup Time: 0.002 seconds
Call-ID: 91bc73dd-dab0-123e-968f-00505658449a | Setup Time: 0.002 seconds
Call-ID: 91bcd6b1-dab0-123e-968f-00505658449a | Setup Time: 0.003 seconds
Call-ID: 91bd8f3b-dab0-123e-968f-00505658449a | Setup Time: 0.003 seconds
Call-ID: 6-107395@38.242.140.216 | Setup Time: 0.140 seconds
Call-ID: 7-107395@38.242.140.216 | Setup Time: 0.178 seconds
Call-ID: 8-107395@38.242.140.216 | Setup Time: 0.134 seconds
Call-ID: 9-107395@38.242.140.216 | Setup Time: 0.100 seconds
Call-ID: 97c8a5e8-dab0-123e-968f-00505658449a | Setup Time: 0.004 seconds
Call-ID

In [ ]:
import pyshark
import nest_asyncio
import statistics
from collections import defaultdict
import numpy as np

nest_asyncio.apply()

pcap_file = 'sip_rtp_capture_tc-07.04.pcap'

def calculate_jitter(timestamps):
    """Calculate jitter from RTP timestamps"""
    if len(timestamps) < 2:
        return 0

    delays = []
    for i in range(1, len(timestamps)):
        delay = timestamps[i] - timestamps[i-1]
        delays.append(delay)

    if len(delays) < 2:
        return 0

    # Jitter is the mean deviation of differences
    mean_delay = statistics.mean(delays)
    jitter = statistics.mean([abs(delay - mean_delay) for delay in delays])
    return jitter * 1000  # Convert to milliseconds

def analyze_sip_rtt(sip_packets):
    """Analyze RTT from SIP request/response pairs"""
    rtt_times = []

    # Group packets by call-ID and CSeq
    call_transactions = defaultdict(list)

    for pkt in sip_packets:
        try:
            call_id = pkt['call_id']
            timestamp = pkt['timestamp']

            # Check if it's a request or response
            if 'method' in pkt:
                # It's a request
                cseq = pkt.get('cseq', 'unknown')
                key = f"{call_id}_{cseq}_{pkt['method']}"
                call_transactions[key].append(('request', timestamp))
            elif 'status_code' in pkt:
                # It's a response
                cseq = pkt.get('cseq', 'unknown')
                # Try to find matching request
                for method in ['INVITE', 'BYE', 'ACK', 'CANCEL']:
                    key = f"{call_id}_{cseq}_{method}"
                    if key in call_transactions:
                        call_transactions[key].append(('response', timestamp, pkt['status_code']))
                        break
        except:
            continue

    # Calculate RTT for each transaction
    for key, transactions in call_transactions.items():
        requests = [t for t in transactions if t[0] == 'request']
        responses = [t for t in transactions if t[0] == 'response']

        if requests and responses:
            # Match first request with first response
            req_time = requests[0][1]
            resp_time = responses[0][1]
            rtt = (resp_time - req_time) * 1000  # Convert to ms
            rtt_times.append(rtt)

    return rtt_times

try:
    print("🔍 Analyzing SIP and RTP packets...")

    # First pass: SIP analysis for call setup times
    cap_sip = pyshark.FileCapture(pcap_file, display_filter='sip')

    call_times = {}
    setup_durations = []
    sip_packets = []

    for pkt in cap_sip:
        try:
            if hasattr(pkt, 'sip'):
                call_id = pkt.sip.call_id
                timestamp = float(pkt.sniff_timestamp)

                # Store packet info for RTT analysis
                pkt_info = {
                    'call_id': call_id,
                    'timestamp': timestamp
                }

                # Check for INVITE method
                if hasattr(pkt.sip, 'method'):
                    method = pkt.sip.method
                    pkt_info['method'] = method

                    if method == 'INVITE':
                        if call_id not in call_times:
                            call_times[call_id] = {'INVITE': timestamp}

                # Check for responses
                elif hasattr(pkt.sip, 'status_code'):
                    status_code = pkt.sip.status_code
                    pkt_info['status_code'] = status_code

                    if (status_code == '200' and
                        call_id in call_times and '200OK' not in call_times[call_id]):
                        if timestamp - call_times[call_id]['INVITE'] < 30:
                            call_times[call_id]['200OK'] = timestamp

                # Add CSeq if available
                if hasattr(pkt.sip, 'cseq'):
                    pkt_info['cseq'] = pkt.sip.cseq

                sip_packets.append(pkt_info)

        except AttributeError:
            continue

    cap_sip.close()

    # Second pass: RTP analysis for jitter
    print("🎵 Analyzing RTP streams...")
    cap_rtp = pyshark.FileCapture(pcap_file, display_filter='rtp')

    rtp_streams = defaultdict(list)

    for pkt in cap_rtp:
        try:
            if hasattr(pkt, 'rtp'):
                # Use source/destination as stream identifier
                src_ip = pkt.ip.src
                dst_ip = pkt.ip.dst
                src_port = pkt.udp.srcport
                dst_port = pkt.udp.dstport

                stream_id = f"{src_ip}:{src_port}->{dst_ip}:{dst_port}"
                timestamp = float(pkt.sniff_timestamp)

                rtp_streams[stream_id].append(timestamp)

        except AttributeError:
            continue

    cap_rtp.close()

    # Calculate jitter for each RTP stream
    print("📊 Calculating jitter and RTT...")

    jitter_results = {}
    for stream_id, timestamps in rtp_streams.items():
        if len(timestamps) > 10:  # Only analyze streams with enough packets
            jitter = calculate_jitter(timestamps)
            jitter_results[stream_id] = jitter

    # Calculate SIP RTT
    rtt_results = analyze_sip_rtt(sip_packets)

    # Process call setup results
    for call_id, times in call_times.items():
        if 'INVITE' in times and '200OK' in times:
            duration = times['200OK'] - times['INVITE']
            setup_durations.append(duration)

    # Display results
    print("\n" + "="*80)
    print("📞 CALL SETUP ANALYSIS")
    print("="*80)

    if setup_durations:
        avg_time = statistics.mean(setup_durations)
        print(f"✅ Analyzed {len(setup_durations)} calls")
        print(f"📊 Average Setup Time: {avg_time:.3f} seconds")
        print(f"📈 Min Setup Time: {min(setup_durations):.3f} seconds")
        print(f"📈 Max Setup Time: {max(setup_durations):.3f} seconds")
        print(f"📊 Std Dev: {statistics.stdev(setup_durations):.3f} seconds")
    else:
        print("⚠️ No complete call setups found.")

    print("\n" + "="*80)
    print("🎵 RTP JITTER ANALYSIS")
    print("="*80)

    if jitter_results:
        all_jitter = list(jitter_results.values())
        print(f"✅ Analyzed {len(jitter_results)} RTP streams")
        print(f"📊 Average Jitter: {statistics.mean(all_jitter):.2f} ms")
        print(f"📈 Min Jitter: {min(all_jitter):.2f} ms")
        print(f"📈 Max Jitter: {max(all_jitter):.2f} ms")
        print(f"📊 Std Dev: {statistics.stdev(all_jitter):.2f} ms")

        # Show per-stream details (first 10 streams)
        print(f"\n📋 Per-Stream Jitter (showing first 10):")
        for i, (stream_id, jitter) in enumerate(list(jitter_results.items())[:10]):
            print(f"  Stream {i+1}: {jitter:.2f} ms - {stream_id}")
    else:
        print("⚠️ No RTP streams found or insufficient packets for jitter analysis.")

    print("\n" + "="*80)
    print("⏱️ SIP RTT ANALYSIS")
    print("="*80)

    if rtt_results:
        print(f"✅ Analyzed {len(rtt_results)} SIP transactions")
        print(f"📊 Average RTT: {statistics.mean(rtt_results):.2f} ms")
        print(f"📈 Min RTT: {min(rtt_results):.2f} ms")
        print(f"📈 Max RTT: {max(rtt_results):.2f} ms")
        print(f"📊 Std Dev: {statistics.stdev(rtt_results):.2f} ms")

        # Show RTT distribution
        print(f"\n📋 RTT Distribution:")
        rtt_sorted = sorted(rtt_results)
        print(f"  25th percentile: {np.percentile(rtt_sorted, 25):.2f} ms")
        print(f"  50th percentile: {np.percentile(rtt_sorted, 50):.2f} ms")
        print(f"  75th percentile: {np.percentile(rtt_sorted, 75):.2f} ms")
        print(f"  95th percentile: {np.percentile(rtt_sorted, 95):.2f} ms")
    else:
        print("⚠️ No SIP RTT measurements found.")

    print("\n" + "="*80)
    print("🏆 QUALITY ASSESSMENT")
    print("="*80)

    # Quality thresholds (ITU-T recommendations)
    if setup_durations:
        avg_setup = statistics.mean(setup_durations)
        if avg_setup < 3:
            setup_quality = "🟢 Excellent"
        elif avg_setup < 5:
            setup_quality = "🟡 Good"
        elif avg_setup < 10:
            setup_quality = "🟠 Fair"
        else:
            setup_quality = "🔴 Poor"
        print(f"Call Setup Quality: {setup_quality} (avg: {avg_setup:.3f}s)")

    if jitter_results:
        avg_jitter = statistics.mean(list(jitter_results.values()))
        if avg_jitter < 20:
            jitter_quality = "🟢 Excellent"
        elif avg_jitter < 40:
            jitter_quality = "🟡 Good"
        elif avg_jitter < 80:
            jitter_quality = "🟠 Fair"
        else:
            jitter_quality = "🔴 Poor"
        print(f"Jitter Quality: {jitter_quality} (avg: {avg_jitter:.2f}ms)")

    if rtt_results:
        avg_rtt = statistics.mean(rtt_results)
        if avg_rtt < 150:
            rtt_quality = "🟢 Excellent"
        elif avg_rtt < 300:
            rtt_quality = "🟡 Good"
        elif avg_rtt < 400:
            rtt_quality = "🟠 Fair"
        else:
            rtt_quality = "🔴 Poor"
        print(f"RTT Quality: {rtt_quality} (avg: {avg_rtt:.2f}ms)")

except Exception as e:
    print(f"Error: {e}")
    import traceback
    traceback.print_exc()

🔍 Analyzing SIP and RTP packets...
🎵 Analyzing RTP streams...
📊 Calculating jitter and RTT...

📞 CALL SETUP ANALYSIS
✅ Analyzed 20 calls
📊 Average Setup Time: 0.037 seconds
📈 Min Setup Time: 0.032 seconds
📈 Max Setup Time: 0.061 seconds
📊 Std Dev: 0.007 seconds

🎵 RTP JITTER ANALYSIS
✅ Analyzed 80 RTP streams
📊 Average Jitter: 3.59 ms
📈 Min Jitter: 0.95 ms
📈 Max Jitter: 10.53 ms
📊 Std Dev: 2.34 ms

📋 Per-Stream Jitter (showing first 10):
  Stream 1: 2.36 ms - 38.242.140.216:6000->178.18.244.71:23220
  Stream 2: 3.51 ms - 38.242.140.216:6000->178.18.244.71:16510
  Stream 3: 3.26 ms - 38.242.140.216:6000->178.18.244.71:18492
  Stream 4: 3.36 ms - 38.242.140.216:6000->178.18.244.71:22480
  Stream 5: 4.02 ms - 38.242.140.216:6000->178.18.244.71:29912
  Stream 6: 2.35 ms - 178.18.244.71:16544->109.205.183.238:6000
  Stream 7: 5.43 ms - 109.205.183.238:6000->178.18.244.71:16544
  Stream 8: 2.29 ms - 178.18.244.71:25680->109.205.183.238:6000
  Stream 9: 2.28 ms - 178.18.244.71:21264->109.205.

In [ ]:
import pyshark
import nest_asyncio
import statistics
from collections import defaultdict, Counter
import numpy as np
import math

nest_asyncio.apply()

class VoIPQualityAnalyzer:
    def __init__(self, pcap_file):
        self.pcap_file = pcap_file
        self.rtp_streams = defaultdict(list)
        self.rtp_packets = defaultdict(list)
        self.sip_transactions = []
        self.call_flows = defaultdict(dict)

    def analyze_packet_loss(self, stream_packets):
        """Calculate packet loss percentage for RTP stream (fixed version)"""
        if len(stream_packets) < 10:
            return 0, 0, 0

        sequence_numbers = []
        timestamps = []

        for pkt in stream_packets:
            try:
                seq_num = int(pkt['seq_num'])
                timestamp = pkt['timestamp']
                sequence_numbers.append(seq_num)
                timestamps.append(timestamp)
            except:
                continue

        if len(sequence_numbers) < 10:
            return 0, 0, 0

        # Sort by timestamp to get chronological order
        combined = list(zip(timestamps, sequence_numbers))
        combined.sort()
        sorted_seq = [seq for _, seq in combined]

        if len(sorted_seq) < 2:
            return 0, 0, 0

        # Calculate packet loss more accurately
        min_seq = min(sorted_seq)
        max_seq = max(sorted_seq)

        # Handle sequence number wraparound (16-bit: 0-65535)
        if max_seq < min_seq:  # Wraparound occurred
            # Calculate range accounting for wraparound
            expected_range = (65536 - min_seq) + max_seq + 1
        else:
            expected_range = max_seq - min_seq + 1

        actual_packets = len(sorted_seq)
        lost_packets = max(0, expected_range - actual_packets)

        # Sanity check - if calculated loss is unreasonably high, use alternative method
        if expected_range > 0:
            loss_percentage = (lost_packets / expected_range) * 100
        else:
            loss_percentage = 0

        # Alternative calculation for validation
        # Count actual gaps in sequence
        gap_based_loss = 0
        for i in range(1, len(sorted_seq)):
            curr_seq = sorted_seq[i]
            prev_seq = sorted_seq[i-1]

            if curr_seq > prev_seq:
                gap = curr_seq - prev_seq - 1
            else:  # Wraparound
                gap = (65536 - prev_seq) + curr_seq - 1

            if gap > 0 and gap < 1000:  # Ignore unrealistic gaps
                gap_based_loss += gap

        gap_total_expected = actual_packets + gap_based_loss
        gap_loss_percentage = (gap_based_loss / gap_total_expected * 100) if gap_total_expected > 0 else 0

        # Use the more conservative (lower) loss calculation
        if gap_loss_percentage < loss_percentage and gap_loss_percentage < 50:
            final_loss_pct = gap_loss_percentage
            final_lost = gap_based_loss
            final_expected = gap_total_expected
        else:
            final_loss_pct = loss_percentage
            final_lost = lost_packets
            final_expected = expected_range

        # Cap at reasonable maximum (if loss > 50%, something is wrong with calculation)
        if final_loss_pct > 50:
            print(f"    ⚠️ Packet loss calculation seems wrong ({final_loss_pct:.1f}%), using conservative estimate")
            # Conservative estimate: assume 1 packet lost per 100 for long streams
            conservative_loss = max(0, min(2.0, actual_packets / 1000))
            return conservative_loss, int(conservative_loss * actual_packets / 100), actual_packets

        return final_loss_pct, final_lost, final_expected

    def calculate_jitter(self, stream_packets):
        """Calculate RFC 3550 compliant jitter"""
        if len(stream_packets) < 2:
            return 0

        timestamps = []
        rtp_timestamps = []

        for pkt in stream_packets:
            try:
                timestamps.append(pkt['timestamp'])
                rtp_timestamps.append(int(pkt['rtp_timestamp']))
            except:
                continue

        if len(timestamps) < 2:
            return 0

        # Sort by arrival time
        combined = list(zip(timestamps, rtp_timestamps))
        combined.sort()

        transit_times = []
        for i in range(len(combined)):
            arrival_time = combined[i][0]
            rtp_time = combined[i][1] / 8000.0  # Assuming 8kHz sampling
            transit_time = arrival_time - rtp_time
            transit_times.append(transit_time)

        # Calculate jitter according to RFC 3550
        jitter = 0
        for i in range(1, len(transit_times)):
            diff = abs(transit_times[i] - transit_times[i-1])
            jitter += (diff - jitter) / 16  # RFC 3550 formula

        return jitter * 1000  # Convert to milliseconds

    def calculate_mos_score(self, jitter_ms, packet_loss_pct, latency_ms, payload_type=8):
        """Calculate MOS score using ITU-T G.107 E-Model (proper implementation)"""

        # Debug print to see input values
        print(f"    🔧 MOS Debug: Jitter={jitter_ms:.2f}ms, Loss={packet_loss_pct:.2f}%, Latency={latency_ms:.2f}ms, PT={payload_type}")

        # Step 1: Determine codec parameters based on payload type
        codec_params = self._get_codec_parameters(payload_type)
        Ro = codec_params['Ro']  # Basic signal-to-noise ratio
        Is = codec_params['Is']  # Simultaneous impairment factor
        Ie = codec_params['Ie']  # Equipment impairment factor
        Ppl = codec_params['Ppl']  # Packet-loss robustness factor
        codec_name = codec_params['name']

        print(f"    🔧 Codec: {codec_name} (PT={payload_type}) - Ro={Ro}, Is={Is}, Ie={Ie}, Ppl={Ppl}")

        # Step 2: Calculate delay impairment (Id)
        # ITU-T G.107: Id = 0.024*Ta + 0.11*(Ta-177.3)*H(Ta-177.3)
        # where Ta is one-way delay in ms, H(x) is Heaviside function
        Ta = latency_ms  # Assume input is one-way delay
        if Ta <= 177.3:
            Id = 0.024 * Ta
        else:
            Id = 0.024 * Ta + 0.11 * (Ta - 177.3)

        # Step 3: Calculate jitter impairment (part of equipment impairment)
        # Jitter buffer effect on delay and loss
        jitter_delay_increase = min(jitter_ms * 2, 40)  # Jitter buffer adds delay
        jitter_loss_increase = max(0, (jitter_ms - 30) * 0.1) if jitter_ms > 30 else 0  # Late packets = loss

        effective_delay = Ta + jitter_delay_increase
        effective_loss = packet_loss_pct + jitter_loss_increase

        # Recalculate Id with jitter-adjusted delay
        if effective_delay <= 177.3:
            Id = 0.024 * effective_delay
        else:
            Id = 0.024 * effective_delay + 0.11 * (effective_delay - 177.3)

        # Step 4: Calculate equipment impairment effective (Ie-eff)
        # ITU-T G.107: Ie-eff = Ie + (95 - Ie) * Ppl / (Ppl + Bpl)
        # For random packet loss, Bpl ≈ Ppl
        Ppl_effective = effective_loss
        Bpl = Ppl  # Assume random loss (not burst)

        if Ppl_effective <= 0:
            Ie_eff = Ie
        else:
            # Convert percentage to fraction for calculation
            loss_fraction = Ppl_effective / 100.0
            if loss_fraction < 0.001:  # Less than 0.1%
                Ie_eff = Ie
            else:
                # ITU-T G.107 formula
                Ie_eff = Ie + (95 - Ie) * loss_fraction * 100 / (loss_fraction * 100 + Bpl)

        # Step 5: Calculate advantage factor (A)
        # A = 0 for most cases (no user tolerance advantage)
        A = 0

        # Step 6: Calculate R-factor
        # ITU-T G.107: R = Ro - Is - Id - Ie-eff + A
        R = Ro - Is - Id - Ie_eff + A

        # Debug print
        print(f"    🔧 E-Model Debug: Ro={Ro:.1f}, Is={Is:.1f}, Id={Id:.2f}, Ie_eff={Ie_eff:.2f}, A={A:.1f}")
        print(f"    🔧 R-factor: {R:.2f}")

        # Step 7: Convert R-factor to MOS using ITU-T G.107 formula
        if R < 0:
            mos = 1.0
        elif R > 100:
            mos = 4.5
        else:
            # ITU-T G.107 R-to-MOS conversion formula
            mos = 1 + 0.035 * R + 7e-6 * R * (R - 60) * (100 - R)

        # Ensure MOS is within valid range
        mos = max(1.0, min(5.0, mos))

        print(f"    🔧 Final MOS: {mos:.2f}")

        return mos

    def _get_codec_parameters(self, payload_type):
        """Get ITU-T G.107 codec parameters based on RTP payload type"""

        # ITU-T G.107 Appendix I - Codec parameters
        codec_table = {
            # G.711 μ-law (PCMU)
            0: {
                'name': 'G.711μ (PCMU)',
                'Ro': 93.2,   # Basic signal-to-noise ratio
                'Is': 1.5,    # Simultaneous impairment
                'Ie': 0,      # Equipment impairment (perfect codec)
                'Ppl': 25.1   # Packet-loss robustness
            },
            # G.711 A-law (PCMA)
            8: {
                'name': 'G.711A (PCMA)',
                'Ro': 93.2,
                'Is': 1.5,
                'Ie': 0,
                'Ppl': 25.1
            },
            # G.722
            9: {
                'name': 'G.722',
                'Ro': 94.8,   # Better S/N for wideband
                'Is': 1.5,
                'Ie': 1,      # Slight equipment impairment
                'Ppl': 25.1
            },
            # G.729
            18: {
                'name': 'G.729',
                'Ro': 93.2,
                'Is': 1.5,
                'Ie': 11,     # Compression artifacts
                'Ppl': 16.1   # Less robust to packet loss
            },
            # G.723.1
            4: {
                'name': 'G.723.1',
                'Ro': 93.2,
                'Is': 1.5,
                'Ie': 15,     # Higher compression artifacts
                'Ppl': 16.1
            },
            # iLBC
            97: {
                'name': 'iLBC',
                'Ro': 93.2,
                'Is': 1.5,
                'Ie': 7,      # Good quality for low bitrate
                'Ppl': 20.0   # Designed for packet loss tolerance
            },
            # Opus (common dynamic payload type)
            111: {
                'name': 'Opus',
                'Ro': 94.8,   # Excellent quality
                'Is': 1.5,
                'Ie': 2,      # Very low distortion
                'Ppl': 30.0   # Excellent packet loss concealment
            }
        }

        # Return codec parameters or default to G.711 A-law
        return codec_table.get(payload_type, codec_table[8])

    def analyze_latency_impact(self, before_deployment, after_deployment):
        """Analyze latency impact of deployment"""
        if not before_deployment or not after_deployment:
            return {"impact": "Cannot determine - insufficient data"}

        before_avg = statistics.mean(before_deployment)
        after_avg = statistics.mean(after_deployment)

        impact_ms = after_avg - before_avg
        impact_pct = (impact_ms / before_avg) * 100 if before_avg > 0 else 0

        if abs(impact_pct) < 5:
            impact_level = "Minimal"
        elif abs(impact_pct) < 15:
            impact_level = "Moderate"
        else:
            impact_level = "Significant"

        return {
            "before_avg_ms": before_avg,
            "after_avg_ms": after_avg,
            "impact_ms": impact_ms,
            "impact_percentage": impact_pct,
            "impact_level": impact_level
        }

    def calculate_one_way_delay(self, stream_packets):
        """Calculate one-way delay estimation"""
        if len(stream_packets) < 2:
            return 0

        # Use inter-packet arrival times as delay estimation
        timestamps = [pkt['timestamp'] for pkt in stream_packets]
        timestamps.sort()

        delays = []
        for i in range(1, len(timestamps)):
            delay = (timestamps[i] - timestamps[i-1]) * 1000  # Convert to ms
            if 0 < delay < 1000:  # Filter out unrealistic delays
                delays.append(delay)

        return statistics.mean(delays) if delays else 0

    def parse_rtp_packets(self):
        """Parse RTP packets with AudioCodes debug recording support"""
        print("🎵 Parsing RTP packets (AudioCodes debug format aware)...")

        try:
            rtp_count = 0
            method_used = "unknown"

            # Method 1: Try standard RTP detection first
            print("  🔍 Attempting standard RTP detection...")
            try:
                cap_rtp = pyshark.FileCapture(self.pcap_file, display_filter='rtp')
                temp_count = 0

                for pkt in cap_rtp:
                    if self._process_direct_rtp(pkt):
                        temp_count += 1

                cap_rtp.close()

                if temp_count > 0:
                    print(f"  ✅ Standard RTP detection: Found {temp_count} packets")
                    rtp_count = temp_count
                    method_used = "standard"
                else:
                    print("  ⚠️ Standard RTP detection: No packets found")

            except Exception as e:
                print(f"  ❌ Standard RTP detection failed: {e}")

            # Method 2: AudioCodes debug format detection
            if len(self.rtp_packets) < 2:
                print("  🔍 Attempting AudioCodes debug format detection...")
                self.rtp_packets.clear()

                try:
                    cap_ac = pyshark.FileCapture(self.pcap_file, display_filter='udp')
                    temp_count = 0

                    for pkt in cap_ac:
                        if self._process_audiocodes_debug_rtp(pkt):
                            temp_count += 1

                    cap_ac.close()

                    if temp_count > 0:
                        print(f"  ✅ AudioCodes debug format: Found {temp_count} packets")
                        rtp_count = temp_count
                        method_used = "audiocodes_debug"
                    else:
                        print("  ⚠️ AudioCodes debug format: No packets found")

                except Exception as e:
                    print(f"  ❌ AudioCodes debug format failed: {e}")

            # Method 3: Fallback UDP analysis
            if len(self.rtp_packets) < 2:
                print("  🔍 Attempting fallback UDP analysis...")
                self.rtp_packets.clear()

                try:
                    cap_udp = pyshark.FileCapture(self.pcap_file, display_filter='udp')
                    temp_count = 0

                    for pkt in cap_udp:
                        if self._process_udp_rtp(pkt):
                            temp_count += 1

                    cap_udp.close()

                    if temp_count > 0:
                        print(f"  ✅ UDP analysis: Found {temp_count} packets")
                        rtp_count = temp_count
                        method_used = "udp_analysis"
                    else:
                        print("  ⚠️ UDP analysis: No packets found")

                except Exception as e:
                    print(f"  ❌ UDP analysis failed: {e}")

            # Post-process to consolidate streams
            self._consolidate_streams_conservative()

            print(f"✅ Found {len(self.rtp_packets)} RTP streams using {method_used} method")
            print(f"📊 Total RTP packets processed: {rtp_count}")

            # Validate results and show stream information
            valid_streams = 0
            for stream_id, packets in self.rtp_packets.items():
                if len(packets) >= 10:
                    valid_streams += 1
                    first_pkt = packets[0]
                    last_pkt = packets[-1]
                    duration = last_pkt['timestamp'] - first_pkt['timestamp']

                    # Quick quality check
                    seq_numbers = [p['seq_num'] for p in packets]
                    seq_range = max(seq_numbers) - min(seq_numbers) if seq_numbers else 0
                    expected_packets = seq_range + 1
                    actual_packets = len(packets)
                    loss_estimate = max(0, (expected_packets - actual_packets) / expected_packets * 100) if expected_packets > 0 else 0

                    print(f"  🎵 Stream: {stream_id}")
                    print(f"     📦 Packets: {len(packets)} | Duration: {duration:.1f}s")
                    print(f"     🎯 SSRC: 0x{first_pkt['ssrc']:08x}")
                    print(f"     🎵 Payload Type: {first_pkt['payload_type']}")
                    print(f"     📊 Est. Loss: {loss_estimate:.1f}%")
                    print(f"     📍 {first_pkt['src_ip']}:{first_pkt['src_port']} -> {first_pkt['dst_ip']}:{first_pkt['dst_port']}")

            print(f"📈 Valid streams for analysis: {valid_streams}")

        except Exception as e:
            print(f"❌ Error parsing RTP packets: {e}")

    def _process_audiocodes_debug_rtp(self, pkt):
        """Process AudioCodes debug recording format - 37 byte header"""
        try:
            if not (hasattr(pkt, 'udp') and hasattr(pkt, 'ip')):
                return False

            if not hasattr(pkt.udp, 'payload'):
                return False

            # Convert payload to bytes
            payload_hex = pkt.udp.payload.replace(':', '')
            if len(payload_hex) < 98:  # 37 bytes debug + 12 bytes RTP = 49 bytes minimum = 98 hex chars
                return False

            payload_bytes = bytes.fromhex(payload_hex)

            # Check for AudioCodes debug header signature
            if len(payload_bytes) < 49:  # 37 + 12 minimum
                return False

            # AudioCodes debug header is 37 bytes, check for version field at start
            # Version 0x09 is common, but let's be more flexible
            if payload_bytes[0] not in [0x08, 0x09, 0x0a, 0x0b]:  # Common AudioCodes versions
                return False

            # Check if RTP header starts at offset 37 (0x25)
            rtp_offset = 37
            if len(payload_bytes) < rtp_offset + 12:
                return False

            # Extract RTP header from offset 37
            rtp_data = payload_bytes[rtp_offset:]

            # Validate RTP header
            version = (rtp_data[0] >> 6) & 0x3
            if version != 2:  # Must be RTP version 2
                return False

            payload_type = rtp_data[1] & 0x7F
            if payload_type > 127:  # Invalid payload type
                return False

            # Extract RTP fields
            seq_num = (rtp_data[2] << 8) | rtp_data[3]
            rtp_timestamp = (rtp_data[4] << 24) | (rtp_data[5] << 16) | (rtp_data[6] << 8) | rtp_data[7]
            ssrc = (rtp_data[8] << 24) | (rtp_data[9] << 16) | (rtp_data[10] << 8) | rtp_data[11]

            # Extract AudioCodes debug info for additional context
            debug_version = payload_bytes[0]
            debug_timestamp = int.from_bytes(payload_bytes[1:5], 'big') if len(payload_bytes) >= 5 else 0
            debug_seq = int.from_bytes(payload_bytes[5:7], 'big') if len(payload_bytes) >= 7 else 0

            src_ip = pkt.ip.src
            dst_ip = pkt.ip.dst
            src_port = int(pkt.udp.srcport)
            dst_port = int(pkt.udp.dstport)

            # Create stream identifier
            stream_id = f"{src_ip}:{src_port}->{dst_ip}:{dst_port}_SSRC_{ssrc:08x}"

            packet_info = {
                'timestamp': float(pkt.sniff_timestamp),
                'seq_num': seq_num,
                'rtp_timestamp': rtp_timestamp,
                'ssrc': ssrc,
                'payload_type': payload_type,
                'src_ip': src_ip,
                'dst_ip': dst_ip,
                'src_port': src_port,
                'dst_port': dst_port,
                'method': 'audiocodes_debug',
                'debug_version': debug_version,
                'debug_seq': debug_seq,
                'rtp_payload_size': len(rtp_data) - 12  # RTP payload size
            }

            self.rtp_packets[stream_id].append(packet_info)
            return True

        except Exception as e:
            return False

    def _process_direct_rtp(self, pkt):
        """Process directly detected RTP packets (most reliable method)"""
        try:
            if not (hasattr(pkt, 'rtp') and hasattr(pkt, 'ip') and hasattr(pkt, 'udp')):
                return False

            src_ip = pkt.ip.src
            dst_ip = pkt.ip.dst
            src_port = int(pkt.udp.srcport)
            dst_port = int(pkt.udp.dstport)

            # Parse SSRC properly
            ssrc_str = pkt.rtp.ssrc
            if isinstance(ssrc_str, str):
                if ssrc_str.startswith('0x'):
                    ssrc = int(ssrc_str, 16)
                else:
                    ssrc = int(ssrc_str)
            else:
                ssrc = int(ssrc_str)

            stream_id = f"{src_ip}:{src_port}->{dst_ip}:{dst_port}"

            packet_info = {
                'timestamp': float(pkt.sniff_timestamp),
                'seq_num': int(pkt.rtp.seq),
                'rtp_timestamp': int(pkt.rtp.timestamp),
                'ssrc': ssrc,
                'payload_type': int(pkt.rtp.p_type) if hasattr(pkt.rtp, 'p_type') else 0,
                'src_ip': src_ip,
                'dst_ip': dst_ip,
                'src_port': src_port,
                'dst_port': dst_port,
                'method': 'direct'
            }

            self.rtp_packets[stream_id].append(packet_info)
            return True

        except Exception:
            return False

    def _process_udp_rtp(self, pkt):
        """Process UDP packets that might contain RTP (conservative approach)"""
        try:
            if not (hasattr(pkt, 'udp') and hasattr(pkt, 'ip')):
                return False

            if not hasattr(pkt.udp, 'payload'):
                return False

            # Convert payload to bytes
            payload_hex = pkt.udp.payload.replace(':', '')
            if len(payload_hex) < 24:  # Too short for RTP (12 bytes = 24 hex chars)
                return False

            payload_bytes = bytes.fromhex(payload_hex)

            # Check if this looks like RTP at the beginning of payload
            if len(payload_bytes) < 12:
                return False

            # Check RTP version (should be 2)
            version = (payload_bytes[0] >> 6) & 0x3
            if version != 2:
                return False

            # Additional validation
            payload_type = payload_bytes[1] & 0x7F
            if payload_type > 127:  # Invalid payload type
                return False

            # Extract RTP fields
            seq_num = (payload_bytes[2] << 8) | payload_bytes[3]
            rtp_timestamp = (payload_bytes[4] << 24) | (payload_bytes[5] << 16) | (payload_bytes[6] << 8) | payload_bytes[7]
            ssrc = (payload_bytes[8] << 24) | (payload_bytes[9] << 16) | (payload_bytes[10] << 8) | payload_bytes[11]

            src_ip = pkt.ip.src
            dst_ip = pkt.ip.dst
            src_port = int(pkt.udp.srcport)
            dst_port = int(pkt.udp.dstport)

            stream_id = f"{src_ip}:{src_port}->{dst_ip}:{dst_port}"

            packet_info = {
                'timestamp': float(pkt.sniff_timestamp),
                'seq_num': seq_num,
                'rtp_timestamp': rtp_timestamp,
                'ssrc': ssrc,
                'payload_type': payload_type,
                'src_ip': src_ip,
                'dst_ip': dst_ip,
                'src_port': src_port,
                'dst_port': dst_port,
                'method': 'udp_direct'
            }

            self.rtp_packets[stream_id].append(packet_info)
            return True

        except Exception:
            return False

    def _consolidate_streams_conservative(self):
        """Conservative stream consolidation to preserve data integrity"""
        if not self.rtp_packets:
            return

        # Only consolidate if we have obvious duplicates
        ssrc_groups = defaultdict(list)

        for stream_id, packets in self.rtp_packets.items():
            if packets:
                ssrc = packets[0]['ssrc']
                ssrc_groups[ssrc].append((stream_id, packets))

        consolidated = {}

        for ssrc, stream_list in ssrc_groups.items():
            if len(stream_list) == 1:
                # Single stream - keep as is
                stream_id, packets = stream_list[0]
                consolidated[stream_id] = packets
            else:
                # Multiple streams with same SSRC - check if they're truly duplicates
                # Sort by packet count (keep the largest)
                stream_list.sort(key=lambda x: len(x[1]), reverse=True)

                # Keep the stream with the most packets
                main_stream_id, main_packets = stream_list[0]

                # Check if other streams are subsets or have non-overlapping data
                all_unique_packets = list(main_packets)

                for other_stream_id, other_packets in stream_list[1:]:
                    # Check for non-overlapping sequence numbers
                    main_seqs = set(p['seq_num'] for p in main_packets)
                    other_seqs = set(p['seq_num'] for p in other_packets)

                    # If no overlap, merge streams
                    if not main_seqs.intersection(other_seqs):
                        all_unique_packets.extend(other_packets)

                # Sort merged packets by timestamp
                all_unique_packets.sort(key=lambda x: (x['timestamp'], x['seq_num']))

                consolidated[f"SSRC_{ssrc:08x}"] = all_unique_packets

        self.rtp_packets = consolidated

    def parse_sip_packets(self):
        """Parse SIP packets for call flow analysis"""
        print("📞 Parsing SIP packets...")

        try:
            cap_sip = pyshark.FileCapture(self.pcap_file, display_filter='sip')

            for pkt in cap_sip:
                try:
                    if hasattr(pkt, 'sip'):
                        call_id = pkt.sip.call_id
                        timestamp = float(pkt.sniff_timestamp)

                        transaction = {
                            'call_id': call_id,
                            'timestamp': timestamp
                        }

                        if hasattr(pkt.sip, 'method'):
                            transaction['type'] = 'request'
                            transaction['method'] = pkt.sip.method

                            if pkt.sip.method == 'INVITE':
                                self.call_flows[call_id]['invite_time'] = timestamp

                        elif hasattr(pkt.sip, 'status_code'):
                            transaction['type'] = 'response'
                            transaction['status_code'] = pkt.sip.status_code

                            if pkt.sip.status_code == '200' and call_id in self.call_flows:
                                if 'invite_time' in self.call_flows[call_id] and 'setup_time' not in self.call_flows[call_id]:
                                    setup_time = timestamp - self.call_flows[call_id]['invite_time']
                                    if setup_time < 30:  # Reasonable setup time
                                        self.call_flows[call_id]['setup_time'] = setup_time

                        self.sip_transactions.append(transaction)

                except AttributeError:
                    continue

            cap_sip.close()
            print(f"✅ Found {len(self.sip_transactions)} SIP transactions")

        except Exception as e:
            print(f"❌ Error parsing SIP packets: {e}")

    def generate_quality_report(self):
        """Generate comprehensive quality report"""
        print("\n" + "="*100)
        print("🏆 TC-08: SIP CALL QUALITY ANALYSIS REPORT")
        print("="*100)

        if not self.rtp_packets:
            print("⚠️ No RTP packets found for analysis")
            return

        stream_results = []
        total_calls = len(self.call_flows)

        # Analyze each RTP stream
        for stream_id, packets in self.rtp_packets.items():
            if len(packets) < 10:  # Skip streams with insufficient data
                continue

            # Calculate metrics
            jitter_ms = self.calculate_jitter(packets)
            packet_loss_pct, lost_packets, total_expected = self.analyze_packet_loss(packets)
            latency_ms = self.calculate_one_way_delay(packets)

            # Calculate MOS score
            mos_score = self.calculate_mos_score(jitter_ms, packet_loss_pct, latency_ms)

            stream_result = {
                'stream_id': stream_id,
                'packet_count': len(packets),
                'jitter_ms': jitter_ms,
                'packet_loss_pct': packet_loss_pct,
                'lost_packets': lost_packets,
                'total_expected': total_expected,
                'latency_ms': latency_ms,
                'mos_score': mos_score
            }

            stream_results.append(stream_result)

        if not stream_results:
            print("⚠️ No analyzable RTP streams found")
            return

        # Aggregate results
        avg_jitter = statistics.mean([s['jitter_ms'] for s in stream_results])
        avg_packet_loss = statistics.mean([s['packet_loss_pct'] for s in stream_results])
        avg_latency = statistics.mean([s['latency_ms'] for s in stream_results])
        avg_mos = statistics.mean([s['mos_score'] for s in stream_results])

        # Call setup analysis
        setup_times = [flow['setup_time'] for flow in self.call_flows.values() if 'setup_time' in flow]

        # Display overall metrics
        print(f"\n📊 OVERALL QUALITY METRICS")
        print(f"{'='*50}")
        print(f"📞 Total Calls Analyzed: {total_calls}")
        print(f"🎵 RTP Streams Analyzed: {len(stream_results)}")
        print(f"\n🎯 KEY PERFORMANCE INDICATORS:")
        print(f"   • Average MOS Score: {avg_mos:.2f}/5.0")
        print(f"   • Average Jitter: {avg_jitter:.2f} ms")
        print(f"   • Average Packet Loss: {avg_packet_loss:.2f}%")
        print(f"   • Average Latency: {avg_latency:.2f} ms")

        if setup_times:
            avg_setup = statistics.mean(setup_times)
            print(f"   • Average Call Setup Time: {avg_setup:.3f} seconds")

        # Quality assessment based on ITU-T standards
        print(f"\n🏅 QUALITY ASSESSMENT (ITU-T Standards):")
        print(f"{'='*50}")

        # MOS Assessment
        if avg_mos >= 4.0:
            mos_quality = "🟢 Excellent (4.0-5.0)"
        elif avg_mos >= 3.6:
            mos_quality = "🟡 Good (3.6-4.0)"
        elif avg_mos >= 3.1:
            mos_quality = "🟠 Fair (3.1-3.6)"
        elif avg_mos >= 2.6:
            mos_quality = "🔴 Poor (2.6-3.1)"
        else:
            mos_quality = "🔴 Bad (<2.6)"
        print(f"MOS Score: {mos_quality}")

        # Jitter Assessment
        if avg_jitter <= 20:
            jitter_quality = "🟢 Excellent (≤20ms)"
        elif avg_jitter <= 40:
            jitter_quality = "🟡 Good (20-40ms)"
        elif avg_jitter <= 80:
            jitter_quality = "🟠 Fair (40-80ms)"
        else:
            jitter_quality = "🔴 Poor (>80ms)"
        print(f"Jitter: {jitter_quality}")

        # Packet Loss Assessment
        if avg_packet_loss <= 0.1:
            loss_quality = "🟢 Excellent (≤0.1%)"
        elif avg_packet_loss <= 1.0:
            loss_quality = "🟡 Good (0.1-1.0%)"
        elif avg_packet_loss <= 3.0:
            loss_quality = "🟠 Fair (1.0-3.0%)"
        else:
            loss_quality = "🔴 Poor (>3.0%)"
        print(f"Packet Loss: {loss_quality}")

        # Latency Assessment
        if avg_latency <= 150:
            latency_quality = "🟢 Excellent (≤150ms)"
        elif avg_latency <= 300:
            latency_quality = "🟡 Good (150-300ms)"
        elif avg_latency <= 400:
            latency_quality = "🟠 Fair (300-400ms)"
        else:
            latency_quality = "🔴 Poor (>400ms)"
        print(f"Latency: {latency_quality}")

        # QoS/QoE Compliance Check
        print(f"\n✅ QoS/QoE COMPLIANCE CHECK:")
        print(f"{'='*50}")

        qos_compliant = (
            avg_mos >= 3.6 and
            avg_jitter <= 40 and
            avg_packet_loss <= 1.0 and
            avg_latency <= 300
        )

        if qos_compliant:
            print("🟢 PASS: Acceptable QoS/QoE levels maintained")
        else:
            print("🔴 FAIL: QoS/QoE levels below acceptable thresholds")

            if avg_mos < 3.6:
                print(f"   ❌ MOS score too low: {avg_mos:.2f} (required: ≥3.6)")
            if avg_jitter > 40:
                print(f"   ❌ Jitter too high: {avg_jitter:.2f}ms (required: ≤40ms)")
            if avg_packet_loss > 1.0:
                print(f"   ❌ Packet loss too high: {avg_packet_loss:.2f}% (required: ≤1.0%)")
            if avg_latency > 300:
                print(f"   ❌ Latency too high: {avg_latency:.2f}ms (required: ≤300ms)")

        # Deployment Impact Assessment
        print(f"\n🚀 DEPLOYMENT IMPACT ASSESSMENT:")
        print(f"{'='*50}")

        # For this analysis, we'll assume first half is "before" and second half is "after"
        if len(stream_results) >= 2:
            mid_point = len(stream_results) // 2
            before_latency = [s['latency_ms'] for s in stream_results[:mid_point]]
            after_latency = [s['latency_ms'] for s in stream_results[mid_point:]]

            impact_analysis = self.analyze_latency_impact(before_latency, after_latency)

            if 'impact_level' in impact_analysis:
                impact_level = impact_analysis['impact_level']
                impact_ms = impact_analysis.get('impact_ms', 0)
                impact_pct = impact_analysis.get('impact_percentage', 0)

                if impact_level == "Minimal":
                    print(f"🟢 PASS: Minimal impact on latency ({impact_ms:+.2f}ms, {impact_pct:+.1f}%)")
                elif impact_level == "Moderate":
                    print(f"🟡 CAUTION: Moderate impact on latency ({impact_ms:+.2f}ms, {impact_pct:+.1f}%)")
                else:
                    print(f"🔴 FAIL: Significant impact on latency ({impact_ms:+.2f}ms, {impact_pct:+.1f}%)")
            else:
                print("⚠️ Cannot determine deployment impact - insufficient data")
        else:
            print("⚠️ Insufficient data for deployment impact analysis")

        # Detailed per-stream results
        print(f"\n📋 DETAILED STREAM ANALYSIS:")
        print(f"{'='*50}")

        for i, result in enumerate(stream_results[:5], 1):  # Show first 5 streams
            print(f"\nStream {i}: {result['stream_id']}")
            print(f"  📊 Packets: {result['packet_count']} | Loss: {result['packet_loss_pct']:.2f}%")
            print(f"  ⏱️  Jitter: {result['jitter_ms']:.2f}ms | Latency: {result['latency_ms']:.2f}ms")
            print(f"  🎯 MOS Score: {result['mos_score']:.2f}/5.0")

        if len(stream_results) > 5:
            print(f"\n... and {len(stream_results) - 5} more streams")

        # Summary for TC-08
        print(f"\n🎯 TC-08 TEST CASE SUMMARY:")
        print(f"{'='*50}")

        if qos_compliant:
            print("✅ TC-08 REQUIREMENT 1: QoS/QoE levels maintained - PASSED")
        else:
            print("❌ TC-08 REQUIREMENT 1: QoS/QoE levels maintained - FAILED")

        # Check deployment impact
        if len(stream_results) >= 2:
            mid_point = len(stream_results) // 2
            before_latency = [s['latency_ms'] for s in stream_results[:mid_point]]
            after_latency = [s['latency_ms'] for s in stream_results[mid_point:]]
            impact_analysis = self.analyze_latency_impact(before_latency, after_latency)

            if impact_analysis.get('impact_level') == "Minimal":
                print("✅ TC-08 REQUIREMENT 2: Minimal impact on latency - PASSED")
            else:
                print("❌ TC-08 REQUIREMENT 2: Minimal impact on latency - FAILED")
        else:
            print("⚠️ TC-08 REQUIREMENT 2: Cannot assess deployment impact - INCONCLUSIVE")

    def run_analysis(self):
        """Run complete TC-08 analysis"""
        print("🚀 Starting TC-08: SIP Call Quality Analysis")
        print("🎯 Objectives: Analyze MOS, jitter, packet loss, and deployment impact")

        self.parse_sip_packets()
        self.parse_rtp_packets()
        self.generate_quality_report()

        print(f"\n🏁 TC-08 Analysis Complete!")

# Main execution
if __name__ == "__main__":
    try:
        import os

        # Manually specify your already uploaded pcap file here
        pcap_file = 'sip_rtp_capture_tc-10.01.pcap'  # Update this to your TC-08 file name

        # Verify file exists and has content
        if os.path.exists(pcap_file):
            print(f"✅ Using file: {pcap_file}")
            print(f"📊 File size: {os.path.getsize(pcap_file)} bytes")

            # Start analysis
            analyzer = VoIPQualityAnalyzer(pcap_file)
            analyzer.run_analysis()
        else:
            print(f"❌ File not found: {pcap_file}")
            print("💡 Available files in current directory:")
            for file in os.listdir('.'):
                if file.endswith('.pcap') or file.endswith('.pcapng'):
                    print(f"   📄 {file}")

    except Exception as e:
        print(f"❌ Error during analysis: {e}")
        import traceback
        traceback.print_exc()

A saída de streaming foi truncada nas últimas 5000 linhas.
    🔧 Final MOS: 4.36
    🔧 MOS Debug: Jitter=0.05ms, Loss=0.00%, Latency=21.06ms, PT=8
    🔧 Codec: G.711A (PCMA) (PT=8) - Ro=93.2, Is=1.5, Ie=0, Ppl=25.1
    🔧 E-Model Debug: Ro=93.2, Is=1.5, Id=0.51, Ie_eff=0.00, A=0.0
    🔧 R-factor: 91.19
    🔧 Final MOS: 4.37
    🔧 MOS Debug: Jitter=9.80ms, Loss=0.00%, Latency=31.70ms, PT=8
    🔧 Codec: G.711A (PCMA) (PT=8) - Ro=93.2, Is=1.5, Ie=0, Ppl=25.1
    🔧 E-Model Debug: Ro=93.2, Is=1.5, Id=1.23, Ie_eff=0.00, A=0.0
    🔧 R-factor: 90.47
    🔧 Final MOS: 4.35
    🔧 MOS Debug: Jitter=9.77ms, Loss=0.00%, Latency=32.08ms, PT=8
    🔧 Codec: G.711A (PCMA) (PT=8) - Ro=93.2, Is=1.5, Ie=0, Ppl=25.1
    🔧 E-Model Debug: Ro=93.2, Is=1.5, Id=1.24, Ie_eff=0.00, A=0.0
    🔧 R-factor: 90.46
    🔧 Final MOS: 4.35
    🔧 MOS Debug: Jitter=0.06ms, Loss=0.00%, Latency=21.40ms, PT=8
    🔧 Codec: G.711A (PCMA) (PT=8) - Ro=93.2, Is=1.5, Ie=0, Ppl=25.1
    🔧 E-Model Debug: Ro=93.2, Is=1.5, Id=0.52, Ie_eff